In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('../../data/post_scores.csv')

In [11]:
df

,user_id,post_id,score
0,5f5a90e2-6601-480c-bc08-7592e06beacd,8e142731-e412-4d06-a560-a93c5de4e316,1
1,0db6b893-14aa-4ae5-8e57-692ca7943ab5,4ccdf4bd-6b8e-4a5c-ab24-3d1eb419a3f0,1
2,0db6b893-14aa-4ae5-8e57-692ca7943ab5,700e6fd2-510c-44ac-afee-11c9868bc378,1
3,0db6b893-14aa-4ae5-8e57-692ca7943ab5,e33ff90e-38ba-4195-bb95-91d1da76836e,1
4,0db6b893-14aa-4ae5-8e57-692ca7943ab5,be64541c-ada2-4794-89b0-aff0ee94545d,1
5,0db6b893-14aa-4ae5-8e57-692ca7943ab5,8e142731-e412-4d06-a560-a93c5de4e316,1
6,0db6b893-14aa-4ae5-8e57-692ca7943ab5,8983c139-7de8-4df9-abe5-e282784b5e7a,3
7,5f5a90e2-6601-480c-bc08-7592e06beacd,700e6fd2-510c-44ac-afee-11c9868bc378,3


In [12]:
from sklearn.preprocessing import LabelEncoder

user_le = LabelEncoder()
df['enc_user_id'] = user_le.fit_transform(df['user_id'])

post_le = LabelEncoder()
df['enc_post_id'] = post_le.fit_transform(df['post_id'])

In [13]:
X = df[['enc_user_id', 'enc_post_id']]
y = df['score']

In [14]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input,Dense,Embedding,Flatten,Input,concatenate,Add
from keras.regularizers import l2

In [16]:
num_users = df['enc_user_id'].nunique()
num_posts = df['enc_post_id'].nunique()
k = 100
l2_lambda = 0.001

In [17]:
u_input = Input((1,), name='user_input')
u = Embedding(num_users, k, name='user_emb')(u_input)
u = Flatten(name='user_flat')(u)
u = Dense(48, activation='relu', name='user_dense')(u)

In [18]:
p_input = Input((1,), name='post_input')
p = Embedding(num_posts, k, name='post_emb')(p_input)
p = Flatten(name='post_flat')(p)
p = Dense(8, activation='relu', name='post_dense')(p)

In [19]:
x = concatenate([u, p], name='concat')
# x = Dropout(0.1, name='drop1')(x)
x = Dense(16, activation='relu', name='dense1')(x)
x = Dense(4, activation='relu', name='dense2')(x)

In [20]:
u_bias = Embedding(num_users, 1, embeddings_regularizer=l2(l2_lambda), name='user_bias_emb')(u_input)
u_bias = Flatten(name='user_bias_flat')(u_bias)

p_bias = Embedding(num_posts, 1, embeddings_regularizer=l2(l2_lambda), name='post_bias_emb')(p_input)
p_bias = Flatten(name='post_bias_flat')(p_bias)

In [21]:
o = concatenate([x, u_bias, p_bias], name='combined_features')
o = Dense(16, activation='relu', name='combined_dense1')(o)
o = Dense(4, activation='relu', name='combined_dense2')(o)
o = Dense(1, activation='linear' , name='output')(o)

In [22]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError

model = Model(inputs=[u_input, p_input], outputs=o)

metrics = [
    MeanSquaredError(name='mse'),
    MeanAbsoluteError(name='mae'),
    RootMeanSquaredError(name='rmse')
]

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(0.001), loss='mean_squared_error', metrics = metrics)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 post_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 user_emb (Embedding)        (None, 1, 100)               200       ['user_input[0][0]']          
                                                                                                  
 post_emb (Embedding)        (None, 1, 100)               600       ['post_input[0][0]']          
                                                                                              

In [15]:
from keras.utils import plot_model

plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
    show_layer_activations=True,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [24]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.1,
    patience=2,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=True
)

In [25]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5',monitor='val_loss', verbose=1, save_best_only=True)

In [26]:
history = model.fit(
    x=[X['enc_user_id'], X['enc_post_id']],
    y=y, epochs=200,
    # validation_data=([X_val['userId'], X_val['movieId']], y_val),
    # callbacks=[early_stopping, model_checkpoint, model_weights_checkpoint]
)

Epoch 1/200
1/1 [==============================] - 0s 305ms/step - loss: 3.0095 - mse: 3.0095 - mae: 1.5032 - rmse: 1.7348
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 2.9931 - mse: 2.9931 - mae: 1.4987 - rmse: 1.7301
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 2.9803 - mse: 2.9803 - mae: 1.4945 - rmse: 1.7264
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 2.9658 - mse: 2.9658 - mae: 1.4897 - rmse: 1.7222
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 2.9514 - mse: 2.9514 - mae: 1.4846 - rmse: 1.7180
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 2.9369 - mse: 2.9369 - mae: 1.4796 - rmse: 1.7137
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 2.9203 - mse: 2.9203 - mae: 1.4740 - rmse: 1.7089
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 2.9037 - mse: 2.9037 - mae: 1.4682 - rmse: 1.7040
Epoch 9/200
1/1 [=============

In [27]:
tf.keras.models.save_model(model, '../../models/posts/recommendations.h5')

/var/folders/8p/bh2hcnvn3bngf4_j4mcz1jx00000gn/T/ipykernel_2217/1618495889.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, '../../models/posts/recommendations.h5')


In [28]:
user_embeddings = {}

counter = 0

for enc_user_id in df['enc_user_id'].unique():
    user_embedding = model.get_layer('user_emb')(np.array([enc_user_id]))
    user_embedding = tf.keras.backend.flatten(user_embedding)
    user_embedding = tf.expand_dims(user_embedding, axis=0)
    user_dense = model.get_layer('user_dense')(user_embedding)
    
    user_embeddings[user_le.inverse_transform([enc_user_id])[0]] = user_dense.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [29]:
post_embeddings = {}

counter = 0

for enc_post_id in df['enc_post_id'].unique():
    post_embedding = model.get_layer('post_emb')(np.array([enc_post_id]))
    post_embedding = tf.keras.backend.flatten(post_embedding)
    post_embedding = tf.expand_dims(post_embedding, axis=0)
    post_dense = model.get_layer('post_dense')(post_embedding)
    
    post_embeddings[post_le.inverse_transform([enc_post_id])[0]] = post_dense.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [30]:
user_bias_embeddings = {}

counter = 0

for enc_user_id in df['enc_user_id'].unique():
    user_bias_embedding = model.get_layer('user_bias_emb')(np.array([enc_user_id]))
    user_bias_embedding = tf.keras.backend.flatten(user_bias_embedding)

    user_bias_embeddings[user_le.inverse_transform([enc_user_id])[0]] = user_bias_embedding.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [31]:
post_bias_embeddings = {}

counter = 0

for enc_post_id in df['enc_post_id'].unique():
    post_bias_embedding = model.get_layer('post_bias_emb')(np.array([enc_post_id]))
    post_bias_embedding = tf.keras.backend.flatten(post_bias_embedding)
    
    post_bias_embeddings[post_le.inverse_transform([enc_post_id])[0]] = post_bias_embedding.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [32]:
import json

with open('../../models/posts/user_embeddings.json', 'w') as f:
    json.dump(user_embeddings, f)

with open('../../models/posts/post_embeddings.json', 'w') as f:
    json.dump(post_embeddings, f)

with open('../../models/posts/user_bias_embeddings.json', 'w') as f:
    json.dump(user_bias_embeddings, f)

with open('../../models/posts/post_bias_embeddings.json', 'w') as f:
    json.dump(post_bias_embeddings, f)

In [33]:
with open('../../models/posts/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../models/posts/post_embeddings.json') as f:
    post_embeddings = json.load(f)
with open('../../models/posts/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../models/posts/post_bias_embeddings.json') as f:
    movie_bias_embeddings = json.load(f)

In [34]:
def predict(user_id, post_id):
    user_embedding = np.array(user_embeddings[user_id])
    post_embedding = np.array(post_embeddings[post_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, post_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    post_bias_embedding = np.array(post_bias_embeddings[post_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    post_bias_embedding = post_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, post_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]

    return predicted_rating.numpy()

In [35]:
def user_recommendations(user_id):
    user_ratings = df[df['user_id'] == user_id]
    user_ratings = df[df['score'] != 1]
    recommendation = df[~df['post_id'].isin(user_ratings['post_id'])][['post_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict(user_id, x['post_id']), axis=1)
    
    final_rec = recommendation.sort_values(by='score_predict', ascending=False)
    return final_rec

In [36]:
rec = user_recommendations('5f5a90e2-6601-480c-bc08-7592e06beacd')
rec.head()

,post_id,score_predict
1,4ccdf4bd-6b8e-4a5c-ab24-3d1eb419a3f0,2.869398
4,be64541c-ada2-4794-89b0-aff0ee94545d,2.511220
3,e33ff90e-38ba-4195-bb95-91d1da76836e,2.460530
0,8e142731-e412-4d06-a560-a93c5de4e316,1.014233


In [37]:
def test_predictions(user_id):
    user_ratings = df[df['user_id'] == user_id]
    
    user_top_ratings = user_ratings.sort_values(by='score', ascending=False)

    comparision = df[df['post_id'].isin(user_ratings['post_id'])][['post_id']].drop_duplicates()
    comparision['score_predict'] = comparision.apply(lambda x: predict(user_id, x['post_id']), axis=1)
    comparision = user_top_ratings.merge(comparision, on='post_id')

    return comparision

In [38]:
comparision = test_predictions('5f5a90e2-6601-480c-bc08-7592e06beacd')
comparision

,user_id,post_id,score,enc_user_id,enc_post_id,score_predict
0,5f5a90e2-6601-480c-bc08-7592e06beacd,700e6fd2-510c-44ac-afee-11c9868bc378,3,1,1,2.978561
1,5f5a90e2-6601-480c-bc08-7592e06beacd,8e142731-e412-4d06-a560-a93c5de4e316,1,1,3,1.014233
